# 10) Counterfactual Evaluation (IPS/DR) + Team-Draft Interleaving

In [ ]:

%%capture
!pip -q install --upgrade pip
!pip -q install datasets transformers sentence-transformers faiss-cpu rank-bm25 torchmetrics scikit-learn lightgbm langdetect unidecode pandas matplotlib tqdm nltk

In [ ]:

import numpy as np
np.random.seed(42)
N=200000
exam_prob = np.clip(np.random.beta(2,8,size=N), 0.01, 0.99)
rel = np.random.binomial(1, 0.15, size=N)
click = (np.random.rand(N) < exam_prob * (0.1 + 0.5*rel)).astype(int)

In [ ]:

ips_ctr = np.mean(click / exam_prob)
mu = 0.1
dr_ctr = np.mean((click - exam_prob*mu)/exam_prob + mu)
print("IPS CTR:", ips_ctr, "| DR CTR:", dr_ctr)

In [ ]:

def interleave(rankA, rankB, k=10):
    seen=set(); out=[]
    a=b=0
    while len(out)<k and (a<len(rankA) or b<len(rankB)):
        if len(out)%2==0:
            while a<len(rankA) and rankA[a] in seen: a+=1
            if a<len(rankA): out.append((rankA[a],"A")); seen.add(rankA[a]); a+=1
        else:
            while b<len(rankB) and rankB[b] in seen: b+=1
            if b<len(rankB): out.append((rankB[b],"B")); seen.add(rankB[b]); b+=1
    return out

winsA=winsB=0; trials=1000
for t in range(trials):
    A = list(np.random.permutation(200)[:20]); B = list(np.random.permutation(200)[:20])
    il = interleave(A,B,10)
    relA = set(A[:10]); relB = set(B[:12])
    clicks = [1 if (d in relA or d in relB) and np.random.rand()<0.2 else 0 for d,_ in il]
    creditA = sum(c for (d,s),c in zip(il,clicks) if s=="A")
    creditB = sum(c for (d,s),c in zip(il,clicks) if s=="B")
    winsA += (creditA>creditB); winsB += (creditB>creditA)
print("Interleaving wins A:", winsA, "B:", winsB)